# Lab | Revisiting Machine Learning Case Study

In this lab, you will use learningSet.csv file which you already have cloned in today's activities.

In [1]:
!pip install xgboost

import pandas as pd
import numpy as np
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pickle

warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVR, SVC
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold, RFE
from xgboost import XGBRegressor, XGBClassifier
from imblearn.under_sampling import TomekLinks


In [20]:
data = pd.read_csv(r'C:\Users\claud\Downloads\learningSet.csv')

In [3]:
data.shape

(95412, 481)

In [4]:
list(data.columns)

['ODATEDW',
 'OSOURCE',
 'TCODE',
 'STATE',
 'ZIP',
 'MAILCODE',
 'PVASTATE',
 'DOB',
 'NOEXCH',
 'RECINHSE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'MDMAUD',
 'DOMAIN',
 'CLUSTER',
 'AGE',
 'AGEFLAG',
 'HOMEOWNR',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'NUMCHLD',
 'INCOME',
 'GENDER',
 'WEALTH1',
 'HIT',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'DATASRCE',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'WEALTH2',
 'GEOCODE',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ET

In [5]:
data[['TARGET_B','TARGET_D']]

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
95407,0,0.0
95408,0,0.0
95409,0,0.0
95410,1,18.0


In [6]:
data.describe()

,ODATEDW,TCODE,DOB,AGE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,...,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2F,CLUSTER2
count,95412.000000,95412.000000,95412.000000,71747.000000,12386.000000,74126.000000,50680.000000,95412.000000,42558.000000,42558.000000,...,95412.000000,85439.000000,85439.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95280.000000
mean,9141.363256,54.223117,2723.602933,61.611649,1.527773,3.886248,5.345699,3.321438,0.152075,0.059166,...,9135.651648,9151.022917,8.093739,13.347786,95778.176959,0.050759,0.793073,0.500618,1.910053,31.533711
std,343.454752,953.844476,2132.241295,16.664199,0.806861,1.854960,2.742490,9.306899,0.470023,0.262078,...,320.394019,294.257260,8.213242,10.769997,55284.596094,0.219506,4.429725,0.500002,1.072749,18.764614
min,8306.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,7211.000000,0.000000,1.285714,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,8801.000000,0.000000,201.000000,48.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,...,8810.000000,8903.000000,4.000000,8.384615,47910.750000,0.000000,0.000000,0.000000,1.000000,15.000000
50%,9201.000000,1.000000,2610.000000,62.000000,1.000000,4.000000,6.000000,0.000000,0.000000,0.000000,...,9201.000000,9204.000000,6.000000,11.636364,95681.500000,0.000000,0.000000,1.000000,2.000000,32.000000
75%,9501.000000,2.000000,4601.000000,75.000000,2.000000,5.000000,8.000000,3.000000,0.000000,0.000000,...,9409.000000,9409.000000,11.000000,15.477955,143643.500000,0.000000,0.000000,1.000000,3.000000,49.000000
max,9701.000000,72002.000000,9710.000000,98.000000,7.000000,7.000000,9.000000,241.000000,6.000000,4.000000,...,9603.000000,9702.000000,1088.000000,1000.000000,191779.000000,1.000000,200.000000,1.000000,4.000000,62.000000


Data imbalance detected between people who did not donate and people who donated, speacially if we are talking about the high amount donors:

In [7]:
len(data[data['TARGET_D'] > 20]['TARGET_D']), len(data[data['TARGET_D'] > 50]['TARGET_D'])

(950, 46)

Here we notice there are no highly correlated columns, which is likely to reflect in a bad fit for linear regression:

In [8]:
data.corr()['TARGET_D'].sort_values(ascending=False)

TARGET_D    1.000000
TARGET_B    0.774232
RAMNT_4     0.268811
RAMNT_21    0.099339
RAMNT_9     0.090168
              ...   
RDATE_3    -0.125194
RDATE_5    -0.220455
RAMNT_5    -0.272147
ADATE_5          NaN
ADATE_15         NaN
Name: TARGET_D, Length: 407, dtype: float64

### 1. Check for null values in all the columns

In [21]:
nulls = pd.DataFrame(data.isna().sum()*100/len(data), columns=['percentage'])
nulls.sort_values('percentage', ascending = False).head(80)

,percentage
RDATE_5,99.990567
RAMNT_5,99.990567
RDATE_3,99.746363
RAMNT_3,99.746363
RDATE_4,99.705488
...,...
ADATE_16,21.343227
ADATE_14,19.774242
ADATE_9,11.785729
ADATE_11,10.923154


### 2. Exclude the following variables by looking at the definitions. Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

- `OSOURCE` - symbol definitions not provided, too many categories
- `ZIP CODE` - we are including state already


In [22]:
drop_list = ['OSOURCE', 'ZIP', 'MAILCODE', 'PVASTATE']
# as there is no 'ZIP CODE' column, I assumed that should be the 'ZIP' column, but we can also add columns 'MAILCODE' and 'PVASTATE'

In [23]:
for x in drop_list:
    print(data[x].value_counts())

MBC    4539
SYN    3563
AML    3430
BHG    3324
IMP    2986
       ... 
BAR       1
BCN       1
AMS       1
BUR       1
SIF       1
Name: OSOURCE, Length: 896, dtype: int64
85351     61
92653     59
85710     54
95608     50
60619     45
          ..
67104-     1
28438      1
76885      1
55364-     1
33602      1
Name: ZIP, Length: 19938, dtype: int64
     94013
B     1399
Name: MAILCODE, dtype: int64
     93954
P     1453
E        5
Name: PVASTATE, dtype: int64


### 3. Identify columns that over 85% missing values

In [24]:
nulls[nulls['percentage'] > 85]

,percentage
NUMCHLD,87.018404
RDATE_3,99.746363
RDATE_4,99.705488
RDATE_5,99.990567
RDATE_6,99.186685
RDATE_7,90.677273
RDATE_10,89.035970
RDATE_13,87.160944
RDATE_15,92.388798
RDATE_17,90.146942


### 4. Remove those columns from the dataframe

In [25]:
nulls_85 = list(nulls[nulls['percentage'] > 85].index)
drop_list.extend(nulls_85)

In [26]:
data.drop(columns = drop_list)

,ODATEDW,TCODE,STATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,0,IL,3712,0,,,,,XXXX,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,1,CA,5202,0,,,,,XXXX,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,1,NC,0,0,,,,,XXXX,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,0,CA,2801,0,,,,,XXXX,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,0,FL,2001,0,X,X,,,XXXX,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,1,AK,0,0,,,,,XXXX,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,1,TX,5001,0,,,,,XXXX,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,1,MI,3801,0,,X,,,XXXX,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,0,CA,4005,0,X,,,,XXXX,...,18.0,1,L,4,F,X,X,X,11.0,A


### 5. Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest

Note that there are a few null values in the column. We will first replace those null values using the code below:

```python
print(categorical['GENDER'].value_counts())
categorical['GENDER'] = categorical['GENDER'].fillna('F')
```

In [27]:
print(data['GENDER'].value_counts())

F    51277
M    39094
      2957
U     1715
J      365
A        2
C        2
Name: GENDER, dtype: int64


In [28]:
data['GENDER'] = data['GENDER'].fillna('F')

In [29]:
print(data['GENDER'].value_counts())

F    51277
M    39094
      2957
U     1715
J      365
A        2
C        2
Name: GENDER, dtype: int64


- As nothing happened I'll check the empty spaces ' '

In [33]:
data['GENDER'] = data['GENDER'].replace(' ', 'F')

In [34]:
print(data['GENDER'].value_counts())

F    54234
M    39094
U     1715
J      365
A        2
C        2
Name: GENDER, dtype: int64


In [68]:
def clean_gender(data, column):
    gender = ['F', 'M']
    genders_to_clean = [i for i in data['GENDER'].unique() if i not in gender]
    data['GENDER'] = data['GENDER'].replace(genders_to_clean, 'other')
    return data

In [69]:
clean_gender(data, data['GENDER'])

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,,,0,0,,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,,,5001,0,,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,,,3801,0,,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,,,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


In [70]:
data['GENDER'].value_counts()

F        54234
M        39094
other     2084
Name: GENDER, dtype: int64